In [1]:
import syft as sy

In [2]:
# client = sy.login(email="info@openmined.org", password="changethis", port=8080)#, url='http://74.248.9.75')

In [2]:
# third party
import numpy as np
from sys import getsizeof
private_high = np.random.rand(125_000_000 * 4)

In [4]:
# numpy_obj = np.random.rand(125_000_000 * 4)

In [5]:
our_bytes = private_high.tobytes()

In [6]:
from syft.serde.recursive import recursive_scheme

import tempfile
def chunk_bytes(
    field_obj, ser_func, field_name: str | int, builder
) -> None:
    data = ser_func(field_obj)
    size_of_data = len(data)
    with tempfile.TemporaryFile() as tmp_file:
        # Write data to a file to save RAM
        tmp_file.write(data)
        tmp_file.seek(0)
        del data
        
        CHUNK_SIZE = int(5.12e8)  # capnp max for a List(Data) field
        list_size = size_of_data // CHUNK_SIZE + 1
        data_lst = builder.init(field_name, list_size)
        for idx in range(list_size):
            bytes_to_read = min(CHUNK_SIZE, size_of_data)
            data_lst[idx] = tmp_file.read(bytes_to_read)
            size_of_data -= CHUNK_SIZE


In [7]:
x = private_high.tobytes()

In [8]:
with open("file", "wb") as f:
    f.write(x) 

KeyboardInterrupt: 

In [ ]:
x = x[100:]

In [ ]:
len(x)

3999999900

In [ ]:
mv = memoryview(x)

In [ ]:
x = mv[100:]

In [ ]:
x.tobytes()

: 

In [ ]:
msg = recursive_scheme.new_message()
msg.init("fieldsName", 10)
msg.init("fieldsData", 10)

<capnp list builder [[], [], [], [], [], [], [], [], [], []]>

In [ ]:
msg.fieldsName[0] = 'test'
chunk_bytes(private_high, lambda x: x.tobytes(), 0, msg.fieldsData)

In [ ]:
len(msg.to_bytes())

4000000360

In [ ]:
private_high.nbytes / (10**9)

4.0

In [ ]:
a = private_high.tobytes()

In [3]:
# stdlib
from typing import cast

# third party
import numpy as np
import pyarrow as pa

# relative
from syft.util.experimental_flags import ApacheArrowCompression
from syft.util.experimental_flags import flags
from syft.serde.deserialize import _deserialize
from syft.serde.serialize import _serialize

def test_arrow_serialize(obj: np.ndarray) -> bytes:
    original_dtype = obj.dtype
    apache_arrow = pa.Tensor.from_numpy(obj=obj)
    sink = pa.BufferOutputStream()
    pa.ipc.write_tensor(apache_arrow, sink)
    buffer = sink.getvalue()
    if flags.APACHE_ARROW_COMPRESSION is ApacheArrowCompression.NONE:
        numpy_bytes = buffer.to_pybytes()
    else:
        numpy_bytes = pa.compress(
            buffer, asbytes=True, codec=flags.APACHE_ARROW_COMPRESSION.value
        )
    dtype = original_dtype.name

    # return cast(bytes, _serialize((numpy_bytes, buffer.size, dtype), to_bytes=True))
    return (numpy_bytes, buffer.size, dtype)

In [4]:
msg = test_arrow_serialize(private_high)

In [5]:
type(msg[0])

bytes

In [5]:
a = sy.serialize(msg[0])

fqn='builtins.bytes'
<function <lambda> at 0x7f3299050fe0> 3754663016


In [6]:
b = sy.serialize(msg[0], to_bytes=True)

fqn='builtins.bytes'
<function <lambda> at 0x7f3299050fe0> 3754663016


In [8]:
c = sy.serialize(msg, to_bytes=True)

fqn='builtins.tuple'
<function serialize_iterable at 0x7f32af029300> 64
fqn='builtins.bytes'
<function <lambda> at 0x7f3299050fe0> 3754663016
fqn='builtins.int'
<function <lambda> at 0x7f3299051bc0> 32
fqn='builtins.str'
<function <lambda> at 0x7f32990518a0> 56


In [5]:
msg_2 = sy.serialize(msg)

fqn='builtins.tuple'
<function serialize_iterable at 0x7021ab12dc60> 64
fqn='builtins.bytes'
<function <lambda> at 0x7021950f8fe0> 3754663498
fqn='builtins.int'
<function <lambda> at 0x7021950f9bc0> 32
fqn='builtins.str'
<function <lambda> at 0x7021950f98a0> 56


In [9]:
type(msg_2)

capnp.lib.capnp._DynamicStructBuilder

In [10]:
def read_bytes(proto):
    with tempfile.TemporaryFile() as tmp_file:
        # Write data to a file to save RAM
        proto.write(tmp_file)
        del proto
        tmp_file.seek(0)
        return tmp_file.read()

In [ ]:
b = read_bytes(msg_2)

In [6]:
type(msg_2)

capnp.lib.capnp._DynamicStructBuilder

In [7]:
x = msg_2.to_bytes()

In [8]:
y = msg_2.to_bytes_packed()

/tmp/ipykernel_185252/1054012271.py:1: UserWarning: This message has already been written once. Be very careful that you're not setting Text/Struct/List fields more than once, since that will cause memory leaks (both in memory and in the serialized data). You can disable this warning by calling the `clear_write_flag` method of this object after every write.
  y = msg_2.to_bytes_packed()


In [6]:
from syft.serde.arrow import arrow_serialize
msg = arrow_serialize(private_high)

fqn='builtins.tuple'
<function serialize_iterable at 0x7ac25a121d00> 64
fqn='builtins.bytes'
<function <lambda> at 0x7ac244050fe0> 3754658671
fqn='builtins.int'
<function <lambda> at 0x7ac244051c60> 32
fqn='builtins.str'
<function <lambda> at 0x7ac244051940> 56


In [3]:
d = sy.serialize(private_high, to_bytes=True)

fqn='numpy.ndarray'
<function numpy_serialize at 0x7ac25a1211c0> 4000000112
fqn='builtins.tuple'
<function serialize_iterable at 0x7ac25a121d00> 64
fqn='builtins.bytes'
<function <lambda> at 0x7ac244050fe0> 3754658671
fqn='builtins.int'
<function <lambda> at 0x7ac244051c60> 32
fqn='builtins.str'
<function <lambda> at 0x7ac244051940> 56


In [ ]:
chunk_bytes

NameError: name 'chunk_bytes' is not defined

In [ ]:
action_object = sy.ActionObject.from_obj(private_high)

In [ ]:
msg = sy.serialize(action_object, to_proto=True)

fqn='syft.service.action.numpy.NumpyArrayObject'
attr_name='id' 40
fqn='syft.types.uid.UID'
attr_name='value' 56
fqn='builtins.bytes'
attr_name='syft_action_data_cache' 4000000112
fqn='numpy.ndarray'
fqn='builtins.tuple'
fqn='builtins.bytes'
fqn='builtins.int'
fqn='builtins.str'
attr_name='syft_action_data_node_id' 16
fqn='builtins.NoneType'
attr_name='syft_action_data_repr_' 140
fqn='builtins.str'
attr_name='syft_action_data_str_' 120
fqn='builtins.str'
attr_name='syft_action_data_type' 424
fqn='builtins.type'
attr_name='syft_blob_storage_entry_id' 16
fqn='builtins.NoneType'
attr_name='syft_client_verify_key' 16
fqn='builtins.NoneType'
attr_name='syft_created_at' 16
fqn='builtins.NoneType'
attr_name='syft_dont_wrap_attrs' 88
fqn='builtins.list'
fqn='builtins.str'
fqn='builtins.str'
attr_name='syft_has_bool_attr' 28
fqn='builtins.bool'
attr_name='syft_history_hash' 32
fqn='builtins.int'
attr_name='syft_node_location' 16
fqn='builtins.NoneType'
attr_name='syft_node_uid' 16
fqn='builtins

In [ ]:
type(msg)

capnp.lib.capnp._DynamicStructBuilder

* 4gb objects
* 6.5gb system
* ~27gb peak serialization  
* ~17gb for serialization

* system ~ 5gb
* object - 5gb
* bytes - 5gb

In [ ]:
action_object.send(client)

Uploading progress: 100%|██████████| 1232/1232 [00:08<00:00, 152.17it/s]


```python
Pointer
```
array([0.76927523, 0.53988129, 0.80223722, ..., 0.05256553, 0.13019749,
       0.89295591])

In [ ]:
client.api.services.action.get(action_object.id)

ConnectionError: Failed to fetch metadata. Response returned with code 502

In [ ]:
assert False

AssertionError: 

In [ ]:
dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client.upload_dataset(dataset_high)

  0%|          | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|██████████| 1/1 [00:33<00:00, 33.17s/it]


**WARNING**: The total size for asset: 'numpy-data' exceeds '512 MB'. This might result in failure to upload dataset. Please contact #support on OpenMined slack for further assistance.


ConnectionError: Failed to fetch metadata. Response returned with code 502

In [ ]:
client.api.node_name

'mynode'

In [ ]:
import syft as sy
from syft.store.blob_storage.seaweedfs import SeaweedFSClientConfig, SeaweedFSClient
from syft.types.blob_storage import CreateBlobStorageEntry
import np

config = SeaweedFSClientConfig(
        host="localhost",  # "http://localhost:8888"
        port=8333,
        access_key="admin",
        secret_key="admin",
        region="us-east-1",
        default_bucket_name="mynode",
        mount_port=4001,
    )
seaweed_client = SeaweedFSClient(config=config)
connection = seaweed_client.connect()

# from 0.1B to 0.5B of elements, it starts to be slow
# serialized data is from 0.8GB to 4GB
mock_high = np.random.random(200_000_000)
print(mock_high.nbytes / (10**9))
ao = sy.ActionObject.from_obj(mock_high)
data = ao.syft_action_data
storage_entry = CreateBlobStorageEntry.from_obj(data)
res = connection.allocate(obj=storage_entry)
print(res)

NameError: name 'np' is not defined